In [1]:
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

client = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,
)
response = client.invoke("hi")
print(response)

content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019aecc7-739a-7e52-ad70-62e2e918fabb-0' usage_metadata={'input_tokens': 2, 'output_tokens': 38, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 28}}


In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
# 동일
# ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an expert in astronomy. Answer the question. <Question>: {input}'))])
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [ ]:
chain = prompt | client
chain.invoke({"input": "지구의 자전 주기는?"})

# RAG - doc loader
- from langchain_community.document_loaders import WebBaseLoader: 웹페이지 문서 로드
- from langchain_community.document_loaders import TextLoader: 텍스트 파일 문서 로드
- from langchain_community.document_loaders import DirectoryLoader: 디렉토리 내 파일 로드
    - unstructured 패키지 필요
```python
from glob import glob
from langchain_community.document_loaders import DirectoryLoader

files = glob(os.path.join('./', '*.txt'))
loader = DirectoryLoader(path='./', glob='*.txt', loader_cls=TextLoader)
data = loader.load()
```
- from langchain_community.document_loaders.csv_loader import CSVLoader: CSV 문서 로드
- from langchain_community.document_loaders import PyPDFLoader: PDF 문서 로드
    - pypdf 패키지 필요

# RAG - text spliter
- 파일에서 텍스트 추출 -> Document 객체 반환

In [21]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/history.txt")
data = loader.load()
page_content = data[0].page_content
type(data[0])

langchain_core.documents.base.Document

# 문서 분리
- CharacterTextSplitter: 구분자 단위로 분리
- RecursiveCharacterTextSplitter: 구분자 리스트(['\n\n', '\n', ' ', ''])를 재귀적으로 분리
    - ref: https://rudaks.tistory.com/entry/langchain-CharacterTextSplitter%E1%84%8B%E1%85%AA-RecursiveCharacterTextSplitter%E1%84%8B%E1%85%B4-%E1%84%8E%E1%85%A1%E1%84%8B%E1%85%B5
    - 동작: 문단 -> 문장 -> 단어 단위로 재귀적으로 분리
    - 사용: 길고 복잡한 텍스트, 문맥 보존이 필요할 때

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="",  # default="\n\n"
    chunk_size=500,
    chunk_overlap=100,
    length_function=len  # 청크 길이(문자열) 구하는 함수
)
texts = text_splitter.split_text(page_content)
len(texts[0])  # 500

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 500(max) 이하 글자로 쪼개짐
    chunk_overlap=100,
    length_function=len
)
texts = text_splitter.split_text(page_content)
len(texts[0])

413

In [30]:
document_list = text_splitter.split_documents(data)

## RAG - Embedding
- embedding: 텍스트 -> 숫자 기반 벡터로 변환하는 작업
- 활용
    - 의미적 유사도 검색
    - 문서 카테고라이징
    - 텍스트 간 유사도 계산
- 임베딩 모델
    - OpenAI
    - HuggingFace
    - Google
- method
    - embed_documents: 문서를 벡터 공간에 임베딩. 대량의 텍스트 데이터를 배치 단위로 처리할 때 활용
    - embed_query: 단일 텍스트 쿼리 -> 벡터 임베딩. 사용자의 검색 쿼리를 임베딩하여 유사한 내용 찾을 떄 활용

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from numpy import dot
from numpy.linalg import norm


def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))


embeddings_model = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')
documents = [
    '안녕하세요!',
    '어! 오랜만이에요',
    '이름이 어떻게 되세요?',
    '날씨가 추워요',
    'Hello LLM!'
]
embeddings = embeddings_model.embed_documents(
    documents
)
embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

0.7284143477609176
0.6911742670933521
0.7742045707881186
0.6099208636976144
0.6828552861515043


In [40]:
from langchain_community.vectorstores import DistanceStrategy, FAISS

vectorstore = FAISS.from_documents(document_list,
                                   embedding=embeddings_model,
                                   distance_strategy=DistanceStrategy.COSINE
                                   )
docs = vectorstore.similarity_search("한국의 역사가 궁금해", k=1)

In [35]:
docs

[Document(id='5729c8a0-198e-47b1-92fe-97908618934a', metadata={'source': 'data/history.txt'}, page_content='한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.\n\n고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.\n\n4세기경, 고구려, 백제, 신라의 삼국이 한반도의 주요 세력으로 부상했습니다. 이 시기는 삼국이 각각 문화와 기술, 무력을 발전시키며 경쟁적으로 성장한 시기로, 한국 역사에서 중요한 전환점을 마련했습니다. 특히 고구려는 북방의 강대국으로 성장하여 중국과도 여러 차례 전쟁을 벌였습니다.')]

In [42]:
# save db in local
vectorstore.save_local("./rag_vectorstore")
# load db
FAISS.load_local("./rag_vectorstore", embeddings_model, allow_dangerous_deserialization=True)

# RAG - Retriever


In [45]:
query = "한국의 역사가 궁금해"
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
retriever.invoke(query)

[Document(id='3dd16cc2-c5ad-49ee-a8d8-c429c318a46f', metadata={'source': 'data/history.txt'}, page_content='한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.\n\n고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.\n\n4세기경, 고구려, 백제, 신라의 삼국이 한반도의 주요 세력으로 부상했습니다. 이 시기는 삼국이 각각 문화와 기술, 무력을 발전시키며 경쟁적으로 성장한 시기로, 한국 역사에서 중요한 전환점을 마련했습니다. 특히 고구려는 북방의 강대국으로 성장하여 중국과도 여러 차례 전쟁을 벌였습니다.')]

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# chain
def format_docs(docs):
    """검색된 문서를 하나의 문자열로 포맷팅"""
    return "\n\n".join(doc.page_content for doc in docs)


template = """당신은 한국 역사 전문가입니다. 주어진 문맥을 기반으로 질문에 답변해주세요.
답변할 수 없는 경우 "주어진 문맥에서 해당 정보를 찾을 수 없습니다"라고 말씀해주세요.

문맥:
{context}

질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
        {
            "context": retriever | format_docs,  # retriever로 문서 검색 후 포맷팅
            "question": RunnablePassthrough()  # 질문을 그대로 전달
        }
        | prompt
        | client
        | StrOutputParser()
)
rag_chain.invoke("한국의 역사가 궁금해")

'한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정으로, 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁을 거쳤으며, 통일 신라와 고려를 지나 조선까지 많은 변화를 겪었습니다.\n\n구체적으로는 기원전 2333년 단군왕검에 의해 세워졌다고 전해지는 고조선이 한국 역사상 최초의 국가입니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어들었습니다.\n\n4세기경에는 고구려, 백제, 신라의 삼국이 한반도의 주요 세력으로 부상했습니다. 이 시기는 삼국이 각각 문화와 기술, 무력을 발전시키며 경쟁적으로 성장한 시기로, 특히 고구려는 북방의 강대국으로 성장하여 중국과도 여러 차례 전쟁을 벌였습니다.'